In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [4]:
from sklearn.model_selection import KFold

In [5]:
from tensorflow.keras.layers import Conv2D

In [6]:
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import concatenate

In [7]:
from tensorflow import keras

In [8]:
from tensorflow.keras.layers import Bidirectional

In [9]:
from tensorflow.keras.models import Model

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation

In [12]:
from tensorflow.python.keras.layers.embeddings import Embedding

In [13]:
import nltk
import string
from nltk.corpus import stopwords
import re
from nltk.stem.snowball import SnowballStemmer

In [14]:
from sklearn.preprocessing import MinMaxScaler

In [41]:
train_df=pd.read_csv('train_limpio_sin_vect_textos.csv')

In [42]:
test_df=pd.read_csv('test_limpio_sin_vect_textos.csv')

In [43]:
sample = pd.read_csv("sample_submission.csv",encoding = "ISO-8859-1")

In [44]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    ## Convert words to lower case and split them
    text = text.lower()
       
    ## Remove stop words (mejor las dejo)
    #text = text.split()
    #stops = set(stopwords.words("english"))
    #text = [w for w in text if not w in stops and len(w) >= 3]  
    #text = " ".join(text)

    ## Clean the text
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #quito caracteres inusuales
    text = re.sub('â' , '', text)  #quito caracteres inusuales
    text = re.sub('ª' , '', text)  #quito caracteres inusuales
    text = re.sub('ã' , '', text)  #quito caracteres inusuales
    text = re.sub('¼' , '', text)  #quito caracteres inusuales
    text = re.sub('\w*\d\w*', '', text)  # quito numeros
    text = re.sub("[^0-9a-zA-Z]+", " ", text)
    
    ## Stemming
    #text = text.split()
    #stemmer = SnowballStemmer('english')
    #stemmed_words = [stemmer.stem(word) for word in text]
    #text = " ".join(stemmed_words)

    return text

In [45]:
train_df['text'] = train_df['text'].replace(r'http\S+', '', regex=True)
test_df['text'] = test_df['text'].replace(r'http\S+', '', regex=True)

In [46]:
train_df['text'] = train_df['text'].apply(lambda x: str(x).lower())
test_df['text'] = test_df['text'].apply(lambda x: str(x).lower())

In [47]:
target = train_df.target

In [49]:
train_df.text

0       our deeds are the reason of this earthquake ma...
1                   forest fire near la ronge sask canada
2       all residents asked to shelter in place are be...
3        people receive wildfires evacuation orders in...
4       just got sent this photo from ruby alaska as s...
                              ...                        
7608    two giant cranes holding a bridge collapse int...
7609    ariaahrary thetawniest the out of control wild...
7610                                 s of volcano hawaii 
7611    police investigating after an ebike collided w...
7612    the latest more homes razed by northern califo...
Name: text, Length: 7613, dtype: object

# Me armo el input de los textos para hacer embedding

In [83]:
#Esto no deberia ser necesario
train_df.text=train_df.text.map(lambda x: clean_text(x))
test_df.text=test_df.text.map(lambda x: clean_text(x))

In [84]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['text'])

train_sequences = tokenizer.texts_to_sequences(train_df['text'])
train_data = pad_sequences(train_sequences, maxlen=180)

In [85]:
test_sequences = tokenizer.texts_to_sequences(test_df['text'])
test_data = pad_sequences(test_sequences, maxlen=180)

In [86]:
EMBEDDING_FILE = 'glove-dataset/glove.twitter.27B.200d.txt' 
#Por ahora lo dejoe n 50, supongo que solo mejora con 200 si complejizo la red nueronal
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, 'r', encoding='utf-8'))

In [87]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((50000, 200)) #50 o 200, segun las dimensiones del objeto que use
for word, index in tokenizer.word_index.items():
    if index > 50000 - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

# Me armo el input de keywords

In [97]:
train_df['keyword'] = train_df['keyword'].apply(lambda x: str(x).lower())
test_df['keyword'] = test_df['keyword'].apply(lambda x: str(x).lower())

In [98]:
train_df.head(50)

,keyword,text,target,cantidad_links,?,@,!,money_related,tiene_fecha,has_year,...,"ÃÃ¸Ã¥ÃÃ¥_T:\ 40\.736324,\-73\.990062-location",Ã¥_:\ \?\?\ ÃÃ\ \?\ :\ \?-location,Ã¥_Ã¥_Los\ Mina\ CityÂÃ£Â¢-location,Ã¥Â¡Ã¥Â¡Midwest\ ÂÃÂ¢ÂÃÂ¢-location,Ã¥Ã\(\?ÂÃÂ¢`\?ÂÃÂ¢Ã¥Â«\)\?\?-location,Ã¥Ã¸\\_\(\?\)_/Ã¥Ã¸-location,longitud_tweet,letras_seguidas,sentimiento,objetividad
0,earthquak,our deeds are the reason of this earthquake ma...,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,70,2,0.000000,0.000000
1,nan,forest fire near la ronge sask canada,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,40,1,0.100000,0.400000
2,nan,all residents asked to shelter in place are be...,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,130,2,-0.018750,0.387500
3,wildfir,people receive wildfires evacuation orders in...,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,60,1,0.000000,0.000000
4,alaska wildfir,just got sent this photo from ruby alaska as s...,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,90,2,0.000000,0.000000
5,rockyfir cafir wildfir,rockyfire update california hwy closed in both...,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,100,1,-0.112500,0.237500
6,flood disast,flood disaster heavy rain causes flash floodin...,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,100,2,-0.200000,0.500000
7,nan,im on top of the hill and i can see a fire in ...,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,60,2,0.500000,0.500000
8,nan,theres an emergency evacuation happening now i...,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,80,2,0.000000,0.000000
9,nan,im afraid that the tornado is coming to our area,1,0,0,0,0,False,False,False,...,False,False,False,False,False,False,50,1,-0.600000,0.900000


In [99]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['keyword'])

train_sequences = tokenizer.texts_to_sequences(train_df['keyword'])
train_keyword_data = pad_sequences(train_sequences, maxlen=50)

In [100]:
test_sequences = tokenizer.texts_to_sequences(test_df['keyword'])
test_keyword_data = pad_sequences(test_sequences, maxlen=50)

# Me armo el input lineal con el resto de los features

In [92]:
train_no_text = train_df.drop(columns=['text','keyword','target'])
test_no_text = test_df.drop(columns=['text','keyword'])

In [93]:
cs = MinMaxScaler()

In [94]:
trainContinuous = cs.fit_transform(train_no_text)
testContinuous = cs.fit_transform(test_no_text)

# Contolo con k-folds

In [70]:
nlp_text_input = keras.Input(shape=(180,), name='nlp_text_input')
nlp_keyword_input = keras.Input(shape=(50,), name='nlp_keyword_input')
meta_input = keras.Input(shape=(train_no_text.shape[1],), name='meta_input')

In [71]:
kf = KFold(n_splits = 5)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [70]:
test_error=[]
#Si agregamdp una dense mejora el resultado
for train_index,test_index in kf.split(train_data):
    x_train_words,x_test_words = train_data[train_index], train_data[test_index]
    x_train_keyword, x_test_keyword = train_keyword_data[train_index], train_keyword_data[test_index]
    x_train_numerical,x_test_numerical = trainContinuous[train_index], trainContinuous[test_index]
    y_train,y_test = train_df['target'][train_index], train_df['target'][test_index]
    
    emb = Embedding(50000, 200, input_length=100, weights=[embedding_matrix], trainable=False)(nlp_text_input)
    drop = Dropout(0.2)(emb)#agregada propia (creo que andaba mejor antes de agregar esto)
    conv = Conv1D(64, 5, activation='relu') (drop) #agregada propia (creo que andaba mejor antes de agregar esto)
    mp = MaxPooling1D(pool_size=4)(conv)#agregada propia (creo que andaba mejor antes de agregar esto)
    nlp_text_out = Bidirectional(LSTM(128))(emb)  #conv antes era emb

    emb_2 = Embedding(50000, 200, input_length=40, weights=[embedding_matrix], trainable=False)(nlp_keyword_input)
    drop_2 = Dropout(0.2)(emb_2)#agregada propia (creo que andaba mejor antes de agregar esto)
    conv_2 = Conv1D(64, 5, activation='relu') (drop_2) #agregada propia (creo que andaba mejor antes de agregar esto)
    mp_2 = MaxPooling1D(pool_size=4)(conv_2)#agregada propia (creo que andaba mejor antes de agregar esto)
    nlp_keyowrd_out = Bidirectional(LSTM(128))(emb_2)  #conv antes era emb

    x = concatenate([nlp_text_out,nlp_keyowrd_out ,meta_input])
    x = Dense(24, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[nlp_text_input , nlp_keyword_input, meta_input], outputs=[x])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit([x_train_words, x_train_keyword, trainContinuous], target, batch_size=128, epochs=3)

    predictions= model.predict([test_data,x_test_numerical,testContinuous])

    bin_pred = []
    for i in predictions:
        if (i<0.5):
            bin_pred.append(0)
        else:
            bin_pred.append(1)

    test_error.append(f1_score(y_test, bin_pred, average='macro'))
    
print('Error de test:',test_error)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/2
6090/6090 [==============================] - 77s 13ms/sample - loss: 0.5607 - acc: 0.7122
Epoch 2/2
6090/6090 [==============================] - 80s 13ms/sample - loss: 0.4615 - acc: 0.7895
Epoch 1/2
6090/6090 [==============================] - 81s 13ms/sample - loss: 0.5561 - acc: 0.7268
Epoch 2/2
6090/6

 # Realizo la prediccion

In [73]:
emb = Embedding(50000, 200, input_length=100, weights=[embedding_matrix], trainable=False)(nlp_text_input)
drop = Dropout(0.2)(emb)#agregada propia (creo que andaba mejor antes de agregar esto)
conv = Conv1D(64, 5, activation='relu') (drop) #agregada propia (creo que andaba mejor antes de agregar esto)
mp = MaxPooling1D(pool_size=4)(conv)#agregada propia (creo que andaba mejor antes de agregar esto)
nlp_text_out = Bidirectional(LSTM(128))(emb)  #conv antes era emb

emb_2 = Embedding(50000, 200, input_length=40, weights=[embedding_matrix], trainable=False)(nlp_keyword_input)
drop_2 = Dropout(0.2)(emb_2)#agregada propia (creo que andaba mejor antes de agregar esto)
conv_2 = Conv1D(64, 5, activation='relu') (drop_2) #agregada propia (creo que andaba mejor antes de agregar esto)
mp_2 = MaxPooling1D(pool_size=4)(conv_2)#agregada propia (creo que andaba mejor antes de agregar esto)
nlp_keyowrd_out = Bidirectional(LSTM(128))(emb_2)  #conv antes era emb

x = concatenate([nlp_text_out,nlp_keyowrd_out ,meta_input])
x = Dense(24, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)
    
model = Model(inputs=[nlp_text_input , nlp_keyword_input, meta_input], outputs=[x])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.fit([train_data, train_keyword_data,trainContinuous], target, batch_size=128, epochs=3)
    
predictions= model.predict([test_data,test_keyword_data,testContinuous])
    
bin_pred = []
for i in predictions:
    if (i<0.5):
        bin_pred.append(0)
    else:
        bin_pred.append(1)


Epoch 1/5
7613/7613 [==============================] - 217s 28ms/sample - loss: 0.5428 - acc: 0.7343
Epoch 2/5
7613/7613 [==============================] - 219s 29ms/sample - loss: 0.4537 - acc: 0.7947
Epoch 3/5
6144/7613 [=======================>......] - ETA: 42s - loss: 0.4307 - acc: 0.8109

KeyboardInterrupt: 

In [ ]:
sample.target = bin_pred

In [131]:
sample.target.value_counts()

0    2079
1    1184
Name: target, dtype: int64

In [133]:
sample.to_csv('neural_network_vectorized_locations_gloved_keywords_no_kfolds_4_epochs.csv', index=False)